<a href="https://colab.research.google.com/github/Suresh18698/Backend/blob/main/engageai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2023-12-13 11:23:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.2’

ngrok-stable-linux- 100%[===================>]  13.21M  23.0MB/s    in 0.6s    

2023-12-13 11:23:15 (23.0 MB/s) - ‘ngrok-stable-linux-amd64.tgz.2’ saved [13856790/13856790]



In [ ]:
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [ ]:
!./ngrok authtoken 2ZU5UXJNFnJHZRd0DsAKPanM97D_3fcs7G3JGNFThJkX98LTR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install pyngrok


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3
import csv
import json
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok


In [ ]:

app = Flask(__name__)

# SQLite database configuration
DATABASE = 'data.db'
run_with_ngrok(app)


# Function to create a connection to the SQLite database
def get_db_connection():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    return conn

# Create the database table if it doesn't exist
def create_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS contacts (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        email TEXT,
                        phone1 TEXT,
                        phone2 TEXT
                    )''')
    conn.commit()
    conn.close()

create_table()  # Ensure the table is created when the app starts


In [ ]:

# Function to handle CSV data and insert into the database
def handle_csv(file):
    filename = secure_filename(file.filename)
    file.save(filename)

    with open(filename, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        conn = get_db_connection()
        cursor = conn.cursor()
        for row in csv_reader:
            cursor.execute('''INSERT INTO contacts (name, email, phone1, phone2)
                              VALUES (?, ?, ?, ?)''', (row['name'], row['email'], row['phone1'], row['phone2']))
        conn.commit()
        conn.close()

# Function to handle JSON data and insert into the database
def handle_json(file):
    data = json.load(file)
    conn = get_db_connection()
    cursor = conn.cursor()
    for item in data:
        cursor.execute('''INSERT INTO contacts (name, email, phone1, phone2)
                          VALUES (?, ?, ?, ?)''', (item['name'], item['phone1'], item['phone2'], item['email'],))
    conn.commit()
    conn.close()


In [ ]:
# Route to the index page
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:

# Route to handle form submission
@app.route('/submit', methods=['POST'])
def submit():
    if request.method == 'POST':
        # Get the uploaded file
        file = request.files['file']

        # Check if the file is JSON or CSV
        if file.filename.endswith('.json'):
            # Process JSON
            handle_json(file)
        elif file.filename.endswith('.csv'):
            # Process CSV
            handle_csv(file)
        else:
            return "File type not supported. Please upload a JSON or CSV file."

        # Retrieve data from the database to display in display.html
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('''SELECT name, email, phone1, phone2 FROM contacts LIMIT 10''')
        contacts = cursor.fetchall()
        conn.close()

        return render_template('display.html', contacts=contacts)


In [ ]:
@app.route('/filter')
def filter_by_name():
    name = request.args.get('name')

    if name:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('''SELECT name, email, phone1, phone2 FROM contacts WHERE name = ?''', (name,))
        filtered_contacts = cursor.fetchall()
        conn.close()

        return render_template('filtered.html', contacts=filtered_contacts)
    else:
        return "Invalid request"



In [ ]:

@app.route('/update', methods=['POST'])
def insert_record():
    if request.method == 'POST':
        name = request.form.get('name')
        email = request.form.get('email')
        phone1 = request.form.get('phone1')
        phone2 = request.form.get('phone2')

        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('''INSERT INTO contacts (name, email, phone1, phone2) VALUES (?, ?, ?, ?)''', (name, email, phone1, phone2))
        conn.commit()

        cursor.execute('''SELECT name, email, phone1, phone2 FROM contacts ORDER BY name ASC''')
        updated_records = cursor.fetchall()
        conn.close()

        return render_template('display.html', contacts=updated_records)


In [ ]:

@app.route('/insert-page')
def gotoinsertpage():
    return render_template('insert.html')

@app.route('/delete-page')
def gotodeletepage():
    return render_template('delete.html')


In [ ]:
@app.route('/delete', methods=['GET', 'POST'])
def deleting():
    if request.method == 'POST':
        name = request.form.get('name')

        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('''DELETE FROM contacts WHERE name = ?''', (name,))
        conn.commit()

        cursor.execute('''SELECT name, email, phone1, phone2 FROM contacts ORDER BY name ASC''')
        updated_records = cursor.fetchall()
        conn.close()

        return render_template('display.html', contacts=updated_records)  # Render index.html with updated data

    return render_template('delete.html')


In [ ]:
@app.route('/all-records')
def getall_records():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('''SELECT name, email, phone1, phone2 FROM contacts ORDER BY name ASC''')
    all_records = cursor.fetchall()
    conn.close()

    return render_template('all_records.html', contacts=all_records)


In [ ]:
if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f6e0-34-106-250-12.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 11:23:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 11:23:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 11:23:53] "POST /submit HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 11:24:00] "GET /filter?name=Sheryl HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 11:24:14] "GET /filter?name=Michelle HTTP/1.1" 200 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 311, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 508, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typical